<h1>LeakyWaveML</h1>
<h3>Binary Slot Schema</h3>
<br>
This code contains the functions and methods used to generate the training data for the binary scheme and to re-run the numerical simulations for the slots generated by the model. Note that COMSOL Multiphysics is required.

In [3]:
import random
import mph
import numpy as np
import matplotlib.pyplot as plt
import os    
import time
from tqdm.notebook import tqdm
import pickle
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tensorflow as tf

path = 'D:/Josh/Fall 2021/ML-LWA/1dsim.mph'

scattering = [1,2,5,6,7,51,53,56,57]

<h3>Helper Functions</h3>

In [4]:
def start_client(path):
    '''
    Starts COMSOL client and initializes API access
    '''
    
    global model
    global pymodel
    global client
    client = mph.start(version = '5.3a')
    pymodel = client.load(path)
    model = pymodel.java
    
def save_model():
    '''
    Saves model
    '''
        
    model.save(path)
    
def generate_image(random_pattern):
    '''
    Arranges slot design into form appropriate for export (flip due to how element numbering works in COMSOL)
    '''
    
    adjusted = [x - 15 for x in random_pattern]
    out = np.flip(np.array([1 if x in adjusted else 0 for x in np.arange(36)]))
    return out
    
def simulate_random():
    '''
    Generates a random slot design so that the number of slots is even and runs its numerical simulation
    '''
    
    global start_count

    grids_of_interest = random.sample(range(15,51),18)
    this_scattering = scattering + grids_of_interest
    
    model.physics("emw").feature("sctr1").selection().set(this_scattering);
    model.physics("emw").feature("ffd1").feature("ffc1").selection().set(grids_of_interest);

    model.geom("geom1").runAll();
    model.mesh().run();
    model.sol("sol1").runAll();
    
    model.result("pg4").set("data", 'dset1')
    model.result("pg4").feature("ff1").run()
    model.result().export("plot1").set("filename", save_str + '.csv')
    model.result().export("plot1").set("plotgroup", "pg4")
    model.result().export("plot1").set("plot", "ff1")
    
    model.result().export("plot1").run()

    return grids_of_interest

def simulate(vals):
    '''
    Similar to simulate_random but accepts a slot design (vals) rather than generating a new one
    '''
    
    global start_count
    grids_of_interest = vals
    this_scattering = scattering + grids_of_interest
#     print(this_scattering)
    
    model.physics("emw").feature("sctr1").selection().set(this_scattering);
    model.physics("emw").feature("ffd1").feature("ffc1").selection().set(grids_of_interest);

    model.geom("geom1").runAll();
    model.mesh().run();
    model.sol("sol1").runAll();
    
    model.result("pg4").set("data", 'dset1')
    model.result("pg4").feature("ff1").run()
    model.result().export("plot1").set("filename", save_str + '.csv')
    model.result().export("plot1").set("plotgroup", "pg4")
    model.result().export("plot1").set("plot", "ff1")
    
    model.result().export("plot1").run()

    return grids_of_interest

<h3>Training Data Generation</h3>

In [ ]:
for instance in tqdm(np.arange(10)):
    img_list = []

    save_str = 'C:/Users/THzAbacus/Documents/Josh/LeakyWaveML/comsol_results/1dconstantslots/' + str(int(time.time()))

    start_client(path)
    start_time = time.time()
    for i in tqdm(np.arange(500)):
        goi = simulate_random()
        array = generate_image(goi)
        img_list.append(array)

    print(time.time() - start_time, 'seconds')

    with open(save_str + '.pkl', 'wb') as file:
        pickle.dump(img_list, file)

In [150]:
model.save('D:/Josh/Fall 2021/ML-LWA/check1d3.mph')

<h3>Validation -- re-running simulations for the slots the model generates</h3>

In [21]:
with open('1d/peaks_to_slot/constant_power/results/test_data.pkl', 'rb') as pkl_file:
    slots, true, preds = pickle.load(pkl_file)

In [128]:
def top_16(slot):

    median = np.sort(slot)[18]
    out = [1 if x >= median else 0 for x in slot]
    return out

In [149]:
img_list = []

save_str = 'C:/Users/THzAbacus/Documents/Josh/LeakyWaveML/comsol_results/validation'

start_client(path)
start_time = time.time()
for i in tqdm(np.arange(500)):
    
    assump = list(np.where(np.flip(top_16(preds[i])))[0] + 15)
    
    goi = simulate(assump)
    array = generate_image(goi)
    img_list.append(array)
    
print(time.time() - start_time, 'seconds')

with open(save_str + '.pkl', 'wb') as file:
    pickle.dump(img_list, file)

  0%|          | 0/500 [00:00<?, ?it/s]

21680.542711496353 seconds
